In [1]:
# Testing notebook

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
import threading

import time
import random


/Users/bohdan/Documents/Programming/Projects/VSCode/Other Projects/Kahoot Joiner/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
# Constants
KAHOOT_URL = "https://kahoot.it/"
GAME_CODE = "1703609"  # Replace with your game code
BASE_NAME = 'my_name'
JOIN_COUNT = 20 # Number of times you want to join
lock = threading.Lock()


# Multithreaded script

In [13]:
def join_game(driver, game_code):
      game_code_input = driver.find_element(by=By.XPATH, value='//*[@id="game-input"]')  # Adjust XPath if needed
      game_code_input.send_keys(game_code)
      game_code_input.send_keys(Keys.ENTER)

def nickname_input(driver, url, game_code, name_suffix):
      player_name_input = driver.find_element(by=By.XPATH, value='//*[@id="nickname"]') 
      player_name_input.send_keys(f"{BASE_NAME}{name_suffix+1}")  
      player_name_input.send_keys(Keys.ENTER)


In [14]:
# Create a new Chrome session
start_time = time.time()
chrome_options = Options()
chrome_options.add_argument("--headless") # Run without rendering Chrome
driver = webdriver.Chrome(options=chrome_options)

# Multithreaded execution using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=JOIN_COUNT) as executor:
    for i in range(JOIN_COUNT):
        driver.execute_script("window.open('');")  # Open a new tab
        driver.switch_to.window(driver.window_handles[-1])  # Switch to the latest tab
        driver.get(KAHOOT_URL)

    for i in range(JOIN_COUNT):
        driver.switch_to.window(driver.window_handles[i+1])
        executor.submit(join_game, driver, GAME_CODE).result()
    for i in range(JOIN_COUNT):
        driver.switch_to.window(driver.window_handles[i+1])
        executor.submit(nickname_input, driver, KAHOOT_URL, GAME_CODE, i+1).result()

end_time = time.time()

# Print the total time taken
print(f"For {JOIN_COUNT} bots, Total time taken: {end_time - start_time} seconds")

For 20 bots, Total time taken: 18.497915983200073 seconds


In [15]:
driver.quit()

# Regular script

In [150]:
def join_kahoot(driver, url, game_code, name_suffix):
    try: 
      driver.get(url)

      # Enter the game code
      # game_code_input = driver.find_element(by=By.XPATH, value='//*[@id="game-input"]')  # Adjust XPath if needed
      # game_code_input.send_keys(game_code)
      # game_code_input.send_keys(Keys.ENTER)
      WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="game-input"]'))
        ).send_keys(game_code, Keys.ENTER)
      
      
      # Wait for the nickname input field to be visible and interactable
      player_name_input = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, '//*[@id="nickname"]'))
      )
      # Increment name with each join
      player_name_input.send_keys(f"{BASE_NAME}{name_suffix+1}")  
      player_name_input.send_keys(Keys.ENTER)

      
      print(f"Successfully joined game as {BASE_NAME}{name_suffix+1}")
    except Exception as e:
       print(f"Unexpected error in thread {name_suffix}: {e} :()")


In [151]:
start_time = time.time()
# Create a new Chrome session
chrome_options = Options()
chrome_options.add_argument("--headless") # Run without rendering Chrome

driver = webdriver.Chrome(options=chrome_options)

for i in range(JOIN_COUNT):
    driver.execute_script("window.open('');")  # Open a new tab
    driver.switch_to.window(driver.window_handles[-1])  # Switch to the latest tab
    join_kahoot(driver, KAHOOT_URL, GAME_CODE, i+1)



end_time = time.time()

# Print the total time taken
print(f"For {JOIN_COUNT} bots, Total time taken: {end_time - start_time} seconds")
driver.quit()
print("Test completed. Quitted.")

Successfully joined game as penis2
Successfully joined game as penis3
Successfully joined game as penis4
Successfully joined game as penis5
Successfully joined game as penis6
Successfully joined game as penis7
Successfully joined game as penis8
Successfully joined game as penis9
Successfully joined game as penis10
Successfully joined game as penis11
Successfully joined game as penis12
Successfully joined game as penis13
Successfully joined game as penis14
Successfully joined game as penis15
Successfully joined game as penis16
Successfully joined game as penis17
Successfully joined game as penis18
Successfully joined game as penis19
Successfully joined game as penis20
Successfully joined game as penis21
For 20 bots, Total time taken: 31.112478017807007 seconds
Test completed. Quitted.
